# lab2_2

In [3]:
import math
import numpy as np
import sympy as sm
from sympy import Derivative, diff
from itertools import groupby
import matplotlib.pyplot as plt
from tabulate import tabulate
from pprint import pprint
import TDMA

from IPython.display import display, Latex, clear_output

In [4]:
sm.init_printing(use_latex='mathjax')
x, y, t, l = sm.symbols('x, y, t, l', real=True)
C1, C2 = sm.symbols("C1, C2", real=True)
c = sm.symbols("c")
u = sm.Function('u')
ux = u(x)
ux1 = Derivative(u(x), x)
ux2 = Derivative(u(x), (x, 2))

h, uup1, uum1, uu0 = sm.symbols('h, u_i+1, u_i-1, u_i')

uup1, uum1, uu0

(uᵢ₊₁, uᵢ₋₁, uᵢ)

In [5]:
zad1 = {
    'a': 0, 'b': 2,
    'k': 0.5, 'T': 0.4,
    'fx': sm.lambdify(x, 1), 'fxt': 2,
    'g1t': sm.lambdify(t, sm.exp(-t)), 'g2t': sm.lambdify(t, sm.exp(-5*t))
    
}

In [10]:
n = 9
m = 20
hh = (zad1['b'] - zad1['a']) / (n-1)
ll = zad1['T'] / (m-1)
bb = zad1['k'] * ll / hh**2
uut1 = (uu0 - y)/(t)
uux2 = (uup1 - 2*uu0 + uum1)/(h**2)
apro = sm.Eq(uut1, zad1['k']*uux2 + zad1['fxt'])
apro2 = sm.Eq(uu0 - bb*(uup1 - 2*uu0 + uum1), y + l*zad1['fxt'])

us = sm.symbols(' '.join([f"u{i}" for i in range(n)]))
list_x = [zad1['a'] + hh*i for i in range(n)]
list_t = [ll*i for i in range(m)]
list_g1t = [zad1['g1t'](list_t[i]) for i in range(m)]
list_g2t = [zad1['g2t'](list_t[i]) for i in range(m)]

print(hh, ll)
display(uut1, uux2)
display(apro, apro2)
print(us)
#print("list_x:", *list_x)
#print("list_tt:", *list_tt)
#print("list_g1t:", *list_g1t)
#print("list_g2t:", *list_g2t)

0.25 0.021052631578947368


uᵢ - y
──────
  t   

-2⋅uᵢ + uᵢ₊₁ + uᵢ₋₁
───────────────────
          2        
         h         

uᵢ - y       0.5⋅(-2⋅uᵢ + uᵢ₊₁ + uᵢ₋₁)
────── = 2 + ─────────────────────────
  t                       2           
                         h            

1.33684210526316⋅uᵢ - 0.168421052631579⋅uᵢ₊₁ - 0.168421052631579⋅uᵢ₋₁ = 2⋅l + 
y

(u0, u1, u2, u3, u4, u5, u6, u7, u8)


In [11]:
table = [[None]*(n+2) for _ in range(m+4)]
table[0][0] = ''
table[0][1] = 'i'
for i in range(2, n+2):
    table[0][i] = i-2
table[1][0] = 'j'
table[1][1] = 'tj \\ xi'
for i in range(2, n+2):
    table[1][i] = list_x[i-2]
    
for i in range(2, m+2):
    table[i][0] = i-2
for i in range(2, m+2):
    table[i][1] = round(list_t[i-2], 3)
table[m+4-2][1] = "й(х;%s)" % table[m+4-3][1]
table[m+4-1][1] = "|й - u|"
for i in range(m):
    table[i+2][2] = list_g1t[i]
    table[i+2][n+2-1] = list_g2t[i]
 
for i in range(1, n-1):
    table[2][i+2] = zad1['fx'](list_x[i])

for i in range(1, m):
    tyts = []
    tyts.append(sm.Eq(us[0], table[2+i][2]))
    tyts.extend([apro2.subs({x: list_x[j], y: table[2+i-1][2+j], h: hh, t: list_t[j],
                             l: ll, uum1: us[j-1], uu0: us[j], uup1: us[j+1]})  for j in range(1,n-1)])
    tyts.append(sm.Eq(us[n-1], table[2+i][2+n-1]))
    
    express = list(map(lambda t: TDMA.maper(t.lhs), tyts))
    TDMA.maper3(express, n)
    expr_values = list(map(lambda t: list(map(lambda t2: TDMA.maper2(t2), t)), express))
    f_values = list(map(lambda t: float(t.rhs), tyts))
    #print(tabulate(expr_values, floatfmt=".3"))
    a = [0] + [expr_values[i+1][i] for i in range(n-1)]
    b = [expr_values[i][i] for i in range(n)]
    c = [expr_values[i][i+1] for i in range(n-1)] + [0]
    xxx = TDMA.TDMA(*[a,b,c], f_values)
    #result = sm.solve(tyts, us)
    #print(tabulate({'U':result.keys(), 'TDMA':xxx, 'SymPy':result.values()}, headers='keys', tablefmt="github" , floatfmt="10.5f"))    #print()
    #display(*result)
    #break
    for j in range(1, n-1):
        table[2+i][2+j] = xxx[j]
        
print(tabulate(table, floatfmt=".4", tablefmt='fancy_grid'))

╒════╤══════════╤════════╤═══════╤═══════╤═══════╤═══════╤═══════╤═══════╤════════╤════════╕
│    │ i        │ 0.0    │ 1.0   │ 2.0   │ 3.0   │ 4.0   │ 5.0   │ 6.0   │ 7.0    │ 8.0    │
├────┼──────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┤
│ j  │ tj \ xi  │ 0.0    │ 0.25  │ 0.5   │ 0.75  │ 1.0   │ 1.25  │ 1.5   │ 1.75   │ 2.0    │
├────┼──────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┤
│ 0  │ 0.0      │ 1.0    │ 1.0   │ 1.0   │ 1.0   │ 1.0   │ 1.0   │ 1.0   │ 1.0    │ 1.0    │
├────┼──────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┤
│ 1  │ 0.021    │ 0.9792 │ 1.034 │ 1.041 │ 1.042 │ 1.042 │ 1.042 │ 1.04  │ 1.024  │ 0.9001 │
├────┼──────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┤
│ 2  │ 0.042    │ 0.9588 │ 1.062 │ 1.081 │ 1.084 │ 1.084 │ 1.083 │ 1.076 │ 1.035  │ 0.8102 │
├────┼──────────┼────────┼───────┼───────┼───────┼───────┼───────┼────

In [ ]:
hello = 'hello'